## Accessing the splat model directly

In [ ]:
import os
import sys
from pathlib import Path

from collab_splats.wrapper import Splatter, SplatterConfig

Set paths to the file for running splats

In [ ]:
base_dir = Path("/workspace/fieldwork-data/")
session_dir = base_dir / "birds/2024-02-06/SplatsSD"

Initialize the configuration and the splatter

In [ ]:
# Make the configuration
splatter_config = SplatterConfig(
    file_path=session_dir / "C0043.MP4",
    method="rade-features",
    frame_proportion=0.25,  # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

The splatter class will internally handle setup and storing of paths from the preprocessing. First, preprocess the data in advance of extracting features

In [ ]:
# This calls COLMAP --> it will take the most amount of time and scales with the number
# of frames extracted from the video
splatter.preprocess()

After the COLMAP files are created, we can train a splatting model

In [ ]:
splatter.extract_features()

### Load the splat model

In [ ]:
from nerfstudio.utils.eval_utils import eval_setup

# Set the current run (grabs the config file)
splatter._select_run()

_, pipeline, _, _ = eval_setup(
    config_path=Path(splatter.config["model_config_path"])
)

In [ ]:
# Grab an item from the dataloader (camera + image)
camera, batch = pipeline.datamanager.next_train(0)

# Pass through the model
outputs = pipeline.model.get_outputs(camera)
